In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

# Function to initialize Selenium WebDriver
def init_driver():
    options = webdriver.ChromeOptions()
    # Uncomment the following line for headless browsing
    # options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    return driver

# Function to load the webpage using Selenium WebDriver
def load_url(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load completely
    return driver.page_source

# Function to scrape data from the DII CM Provisional page
def scrape_dii_activity_cm(driver, url):
    # Load the webpage
    page_source = load_url(driver, url)

    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(page_source, "html.parser")

    # Find the data table rows
    data_rows = soup.find_all("ion-row", {"class": "ng-star-inserted"})

    if not data_rows:
        print("No data table found. Check the selectors.")
        return

    extracted_data = []

    # Extract data row by row
    for row in data_rows:
        columns = row.find_all("ion-col")
        if len(columns) > 0:
            date = columns[0].text.strip() if len(columns) > 0 else "N/A"
            gross_purchase = columns[1].text.strip() if len(columns) > 1 else "N/A"
            gross_sales = columns[2].text.strip() if len(columns) > 2 else "N/A"
            net_purchase = columns[3].text.strip() if len(columns) > 3 else "N/A"

            extracted_data.append([date, gross_purchase, gross_sales, net_purchase])

    return extracted_data

# Function to save data to a CSV file
def save_to_csv(data, filename="dii_activity_cm_provisional.csv"):
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        # Write header row
        writer.writerow(["Date", "Gross Purchase", "Gross Sales", "Net Purchase"])
        # Write data rows
        writer.writerows(data)

# Main function to execute the scraping
def main():
    # URL of the webpage to scrape
    url = "https://web.stockedge.com/fii-activity?section=cm-provisional&fii-dii-type=dii"

    # Initialize the Selenium WebDriver
    driver = init_driver()

    try:
        # Scrape data from the DII CM Provisional page
        data = scrape_dii_activity_cm(driver, url)

        # Save the scraped data to a CSV file
        save_to_csv(data)
        print("Data has been successfully saved to 'dii_activity_cm_provisional.csv'")
    except Exception as e:
        print("An error occurred:", e)
    finally:
        # Close the driver
        driver.quit()

# Entry point for script execution
if __name__ == "__main__":
    main()
